<a href="https://colab.research.google.com/github/Emmanuel10701/Pandas/blob/main/Python_Email_Slot_Notifier_with_Web_Check_(Conceptual).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import imaplib
import email
import smtplib
import time
import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Import Selenium components
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException

# --- Configuration ---
# Your email account details (the one you want to monitor)
EMAIL_ACCOUNT = os.getenv('MONITOR_EMAIL_ACCOUNT', 'your_monitoring_email@example.com')
EMAIL_PASSWORD = os.getenv('MONITOR_EMAIL_PASSWORD', 'your_monitoring_email_password')
IMAP_SERVER = 'imap.gmail.com' # Or your email provider's IMAP server
IMAP_PORT = 993

# Your email account details (the one you want to send notifications from)
SENDER_EMAIL = os.getenv('SENDER_EMAIL_ACCOUNT', 'your_sending_email@example.com')
SENDER_PASSWORD = os.getenv('SENDER_EMAIL_PASSWORD', 'your_sending_email_password')
SMTP_SERVER = 'smtp.gmail.com' # Or your email provider's SMTP server
SMTP_PORT = 587 # Or 465 for SSL

# Recipient for notifications
NOTIFICATION_RECIPIENT = os.getenv('NOTIFICATION_RECIPIENT', 'your_notification_recipient@example.com')

# Keywords to look for in email subject or body (case-insensitive)
KEYWORDS = ['slot released', 'appointment available', 'booking open']

# How often to check for new emails (in seconds)
CHECK_INTERVAL_SECONDS = 60

# --- Web Scraping Configuration ---
# URL of the US Embassy appointment page you want to monitor
EMBASSY_URL = os.getenv('EMBASSY_URL', 'https://evisaforms.state.gov/default.asp?postcode=NRB&appcode=3') # Example URL for Nairobi
# Path to your ChromeDriver executable
# Download from: https://chromedriver.chromium.org/downloads
CHROMEDRIVER_PATH = os.getenv('CHROMEDRIVER_PATH', '/path/to/chromedriver') # IMPORTANT: Update this path!

# Keywords or phrases to look for on the embassy page indicating slot availability
WEB_AVAILABILITY_KEYWORDS = ['appointments available', 'open slots', 'schedule now', 'next available']
# Elements to check for availability (e.g., a specific div, span, or button text)
# This will be highly specific to the website's HTML structure.
# You'll need to inspect the page to find appropriate selectors.
AVAILABILITY_CHECK_ELEMENT_XPATH = "//div[@id='appointment_status']" # Example XPath, replace with actual

# --- Global variable to store UIDs of processed emails and web status to avoid re-notifying ---
processed_uids = set()
last_web_status_available = False # To prevent repeated web notifications

def connect_to_imap(email_account, email_password, imap_server, imap_port):
    """Establishes a secure IMAP connection and logs in."""
    try:
        mail = imaplib.IMAP4_SSL(imap_server, imap_port)
        mail.login(email_account, email_password)
        print(f"Successfully connected to IMAP server for {email_account}")
        return mail
    except imaplib.IMAP4.error as e:
        print(f"IMAP connection or login failed: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during IMAP connection: {e}")
        return None

def search_emails(mail, keywords):
    """Searches for emails containing any of the specified keywords."""
    try:
        mail.select('inbox') # Select the inbox

        # Search for all emails
        status, email_ids = mail.search(None, 'ALL')
        if status != 'OK':
            print(f"Error searching emails: {status}")
            return []

        email_id_list = email_ids[0].split()
        new_matches = []

        for uid in email_id_list:
            if uid.decode() in processed_uids:
                continue # Skip already processed emails

            status, msg_data = mail.fetch(uid, '(RFC822)')
            if status != 'OK':
                print(f"Error fetching email {uid}: {status}")
                continue

            msg = email.message_from_bytes(msg_data[0][1])
            subject = msg['subject'] if msg['subject'] else ''
            body = ""

            # Decode email body
            if msg.is_multipart():
                for part in msg.walk():
                    ctype = part.get_content_type()
                    cdispo = str(part.get('Content-Disposition'))

                    # Look for plain text parts, not attachments
                    if ctype == 'text/plain' and 'attachment' not in cdispo:
                        try:
                            body = part.get_payload(decode=True).decode()
                            break
                        except UnicodeDecodeError:
                            body = part.get_payload(decode=True).decode('latin-1', errors='ignore')
            else:
                try:
                    body = msg.get_payload(decode=True).decode()
                except UnicodeDecodeError:
                    body = msg.get_payload(decode=True).decode('latin-1', errors='ignore')

            # Check for keywords in subject or body (case-insensitive)
            for keyword in keywords:
                if keyword.lower() in subject.lower() or keyword.lower() in body.lower():
                    new_matches.append({
                        'uid': uid.decode(),
                        'subject': subject,
                        'from': msg['from'],
                        'date': msg['date']
                    })
                    processed_uids.add(uid.decode()) # Mark as processed
                    break # Found a keyword, no need to check others for this email

        return new_matches

    except imaplib.IMAP4.error as e:
        print(f"IMAP search error: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred during email search: {e}")
        return []

def send_email_notification(sender_email, sender_password, smtp_server, smtp_port, recipient, subject, body):
    """Sends an email notification."""
    try:
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = recipient
        msg['Subject'] = subject

        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls() # Secure the connection
            server.login(sender_email, sender_password)
            server.send_message(msg)
        print(f"Notification email sent to {recipient} successfully.")
    except smtplib.SMTPAuthenticationError:
        print("SMTP authentication failed. Check sender email and password.")
    except smtplib.SMTPConnectError as e:
        print(f"SMTP connection failed: {e}. Check SMTP server and port.")
    except Exception as e:
        print(f"An unexpected error occurred during email sending: {e}")

def check_embassy_website(url, driver_path, keywords, element_xpath):
    """
    Checks the embassy website for slot availability using Selenium.
    Returns True if keywords are found, False otherwise.
    """
    options = webdriver.ChromeOptions()
    options.add_argument('--headless') # Run in headless mode (no browser UI)
    options.add_argument('--disable-gpu') # Recommended for headless
    options.add_argument('--no-sandbox') # Required for some environments
    options.add_argument('--log-level=3') # Suppress verbose logging
    options.add_argument('--incognito') # Use incognito mode

    driver = None
    try:
        service = Service(executable_path=driver_path)
        driver = webdriver.Chrome(service=service, options=options)
        print(f"Checking URL: {url}")
        driver.get(url)

        # Wait for the specific element to be present
        # Adjust the timeout as needed
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, element_xpath))
        )

        # Get the text content of the relevant element or the entire page source
        # You might need to adjust this based on where the availability info is.
        # For example, if it's in a specific div:
        element = driver.find_element(By.XPATH, element_xpath)
        page_content = element.text # Or driver.page_source for the whole page

        # Check for keywords in the page content
        for keyword in keywords:
            if keyword.lower() in page_content.lower():
                print(f"Keyword '{keyword}' found on the page!")
                return True

        print("No availability keywords found on the page.")
        return False

    except TimeoutException:
        print(f"Timeout waiting for element {element_xpath} on {url}. Page might not have loaded correctly or element not found.")
        return False
    except WebDriverException as e:
        print(f"WebDriver error: {e}. Ensure ChromeDriver path is correct and Chrome is installed.")
        return False
    except Exception as e:
        print(f"An unexpected error occurred during web check: {e}")
        return False
    finally:
        if driver:
            driver.quit() # Always close the browser

def main():
    global last_web_status_available # Declare global to modify it

    print("Starting email slot notifier...")
    print("Ensure all necessary environment variables are set or hardcoded values are updated.")
    print("Remember to download ChromeDriver and update CHROMEDRIVER_PATH.")

    # Initial IMAP connection attempt
    mail_client = connect_to_imap(EMAIL_ACCOUNT, EMAIL_PASSWORD, IMAP_SERVER, IMAP_PORT)
    if not mail_client:
        print("Exiting due to initial IMAP connection failure.")
        return

    while True:
        try:
            # --- Email Check ---
            # Reconnect IMAP if needed
            if not mail_client or mail_client.state != 'SELECTED':
                print("Reconnecting to IMAP server...")
                mail_client = connect_to_imap(EMAIL_ACCOUNT, EMAIL_PASSWORD, IMAP_SERVER, IMAP_PORT)
                if not mail_client:
                    print("Failed to reconnect IMAP. Retrying in next interval...")
                    time.sleep(CHECK_INTERVAL_SECONDS)
                    continue

            new_emails = search_emails(mail_client, KEYWORDS)

            if new_emails:
                print(f"Found {len(new_emails)} new matching email(s).")
                for email_info in new_emails:
                    notification_subject = f"SLOT ALERT (Email): {email_info['subject']}"
                    notification_body = (
                        f"A new slot-related email has been detected!\n\n"
                        f"From: {email_info['from']}\n"
                        f"Subject: {email_info['subject']}\n"
                        f"Date: {email_info['date']}\n\n"
                        f"Please check your inbox for details."
                    )
                    send_email_notification(SENDER_EMAIL, SENDER_PASSWORD, SMTP_SERVER, SMTP_PORT,
                                            NOTIFICATION_RECIPIENT, notification_subject, notification_body)
            else:
                print("No new matching emails found.")

            # --- Web Check ---
            print("\nInitiating web check...")
            is_available_on_web = check_embassy_website(EMBASSY_URL, CHROMEDRIVER_PATH, WEB_AVAILABILITY_KEYWORDS, AVAILABILITY_CHECK_ELEMENT_XPATH)

            if is_available_on_web and not last_web_status_available:
                print("Web check indicates slots are now available!")
                web_notification_subject = "SLOT ALERT (Web): Appointments Available on Embassy Site!"
                web_notification_body = (
                    f"The US Embassy website ({EMBASSY_URL}) appears to have available slots.\n\n"
                    f"Please check the website immediately to book your appointment!"
                )
                send_email_notification(SENDER_EMAIL, SENDER_PASSWORD, SMTP_SERVER, SMTP_PORT,
                                        NOTIFICATION_RECIPIENT, web_notification_subject, web_notification_body)
                last_web_status_available = True # Set flag to true
            elif not is_available_on_web and last_web_status_available:
                print("Web check indicates slots are no longer available.")
                last_web_status_available = False # Reset flag
            elif is_available_on_web and last_web_status_available:
                print("Slots still available on web (already notified).")
            else:
                print("Slots still not available on web.")


            print(f"\nWaiting {CHECK_INTERVAL_SECONDS} seconds before next check...")
            time.sleep(CHECK_INTERVAL_SECONDS)

        except Exception as e:
            print(f"An error occurred in the main loop: {e}. Restarting check in {CHECK_INTERVAL_SECONDS} seconds.")
            # Attempt to close and re-open IMAP connection on error
            try:
                if mail_client:
                    mail_client.logout()
            except Exception as logout_e:
                print(f"Error during IMAP logout: {logout_e}")
            mail_client = None # Force reconnection in next loop iteration
            time.sleep(CHECK_INTERVAL_SECONDS)

if __name__ == "__main__":
    main()